# Orchestrating Jobs, Model Registration, and Continuous Deployment with Amazon SageMaker

Amazon SageMaker offers Machine Learning application developers and Machine Learning operations engineers the ability to orchestrate SageMaker jobs and author reproducible Machine Learning pipelines, deploy custom-build models for inference in real-time with low latency or offline inferences with Batch Transform, and track lineage of artifacts. You can institute sound operational practices in deploying and monitoring production workflows, deployment of model artifacts, and track artifact lineage through a simple interface, adhering to safety and best-practice paradigmsfor Machine Learning application development.

The SageMaker Pipelines service supports a SageMaker Machine Learning Pipeline Domain Specific Language (DSL), which is a declarative Json specification. This DSL defines a Directed Acyclic Graph (DAG) of pipeline parameters and SageMaker job steps. The SageMaker Python Software Developer Kit (SDK) streamlines the generation of the pipeline DSL using constructs that are already familiar to engineers and scientists alike.

The SageMaker Model Registry is where trained models are stored, versioned, and managed. Data Scientists and Machine Learning Engineers can compare model versions, approve models for deployment, and deploy models from different AWS accounts, all from a single Model Registry. SageMaker enables customers to follow the best practices with ML Ops and getting started right. Customers are able to standup a full ML Ops end-to-end system with a single API call.

## SageMaker Pipelines

Amazon SageMaker Pipelines support the following activites:

* Pipelines - A Directed Acyclic Graph of steps and conditions to orchestrate SageMaker jobs and resource creation.
* Processing Job steps - A simplified, managed experience on SageMaker to run data processing workloads, such as feature engineering, data validation, model evaluation, and model interpretation.
* Training Job steps - An iterative process that teaches a model to make predictions by presenting examples from a training dataset.
* Conditional step execution - Provides conditional execution of branches in a pipeline.
* Registering Models - Creates a model package resource in the Model Registry that can be used to create deployable models in Amazon SageMaker.
* Creating Model steps - Create a model for use in transform steps or later publication as an endpoint.
* Parameterized Pipeline executions - Allows pipeline executions to vary by supplied parameters.
* Transform Job steps - A batch transform to preprocess datasets to remove noise or bias that interferes with training or inference from your dataset, get inferences from large datasets, and run inference when you don't need a persistent endpoint.

## Layout of the SageMaker ModelBuild Project Template

The template provides a starting point for bringing your SageMaker Pipeline development to production.

```
|-- codebuild-buildspec.yml
|-- CONTRIBUTING.md
|-- pipelines
|   |-- abalone
|   |   |-- evaluate.py
|   |   |-- __init__.py
|   |   |-- pipeline.py
|   |   `-- preprocess.py
|   |-- get_pipeline_definition.py
|   |-- __init__.py
|   |-- run_pipeline.py
|   |-- _utils.py
|   `-- __version__.py
|-- README.md
|-- sagemaker-pipelines-project.ipynb
|-- setup.cfg
|-- setup.py
|-- tests
|   `-- test_pipelines.py
`-- tox.ini
```

A description of some of the artifacts is provided below:
<br/><br/>
Your codebuild execution instructions:
```
|-- codebuild-buildspec.yml
```
<br/><br/>
Your pipeline artifacts, which includes a pipeline module defining the required `get_pipeline` method that returns an instance of a SageMaker pipeline, a preprocessing script that is used in feature engineering, and a model evaluation script to measure the Mean Squared Error of the model that's trained by the pipeline:

```
|-- pipelines
|   |-- kg
|   |   |-- __init__.py
|   |   |-- database.py
|   |   |-- evaluate.py
|   |   |-- inference.py
|   |   |-- pipeline.py
|   |   |-- preprocess.py
|   |   |-- train.py
|   |   |-- utils.py

```
<br/><br/>
Utility modules for getting pipeline definition jsons and running pipelines:

```
|-- pipelines
|   |-- get_pipeline_definition.py
|   |-- __init__.py
|   |-- run_pipeline.py
|   |-- _utils.py
|   `-- __version__.py
```
<br/><br/>
Python package artifacts:
```
|-- setup.cfg
|-- setup.py
```
<br/><br/>
A stubbed testing module for testing your pipeline as you develop:
```
|-- tests
|   `-- test_pipelines.py
```
<br/><br/>
The `tox` testing framework configuration:
```
`-- tox.ini
```

### A SageMaker Pipeline

The pipeline that we create follows a typical Machine Learning Application pattern of pre-processing, training, evaluation, and conditional model registration and publication, if the quality of the model is sufficient.

![A typical ML Application pipeline](img/pipeline-full.png)

### Getting some constants

We get some constants from the local execution environment.

In [ ]:
!pip install -U sagemaker

Specify your `default_bucket` or use `sagemaker.session.Session().default_bucket()`.

In [2]:
import boto3
import sagemaker


region = boto3.Session().region_name
role = sagemaker.get_execution_role()
# default_bucket = sagemaker.session.Session().default_bucket()
default_bucket = 'sm-nlp-data'
print(sagemaker.__version__)

2.59.2


In [3]:
role

'arn:aws:iam::093729152554:role/service-role/AWSNeptuneNotebookRole-NepTestRole'

### Upload data for the knowledge graph pipeline.

Copy training data 

In [2]:
%%bash -s "$default_bucket"

wget http://dataset-bj.cdn.bcebos.com/qianyan/DuIE_2_0.zip
aws s3 cp DuIE_2_0.zip "s3://$1/ie-baseline/raw/DuIE_2_0.zip"
rm DuIE_2_0.zip

Process is terminated.


In [4]:
!aws s3 cp ./pipelines/kg/data/psudo_transform_input.json s3://$default_bucket/psudo/psudo.json

upload: pipelines/kg/data/psudo_transform_input.json to s3://sm-nlp-data/psudo/psudo.json


Check whether data exists at the desired location, which would be used later.

In [4]:
raw_input_data_s3_uri = f"s3://{default_bucket}/ie-baseline/raw/DuIE_2_0.zip"
!aws s3 ls $raw_input_data_s3_uri

2021-09-15 06:47:55   37097755 DuIE_2_0.zip


### Get the pipeline instance

Here we get the pipeline instance from your pipeline module so that we can work with it.

In [5]:
from pipelines.kg.pipeline import get_pipeline


pipeline = get_pipeline(
    region=region,
    role=role,
    default_bucket=default_bucket,
    model_package_group_name='KGModelPackageGroup',
    pipeline_name='KGPipeline',
)

BASE_DIR: /root/ckgqa-p-kiqtyrraeiec/sagemaker-ckgqa-p-kiqtyrraeiec-modelbuild/pipelines/kg
SM role ARN: None
Step process created!
Step train created!
Step evaluate created!
Step register model created!
Step create model created!
Step transform created!
ProcessingStep(name='RetrieveOrCreateNeptuneDB', display_name=None, description=None, step_type=<StepTypeEnum.PROCESSING: 'Processing'>, depends_on=None)
TransformStep(name='KgTransform', display_name=None, description=None, step_type=<StepTypeEnum.TRANSFORM: 'Transform'>, depends_on=None)
Step create database created!
Step bulkload created!
Step alert created!
Step condition created!
Pipeline created


### Submit the pipeline to SageMaker and start execution

Let's submit our pipeline definition to the workflow service. The role passed in will be used by the workflow service to create all the jobs defined in the steps. `upsert` update or insert parameters, and then create the pipeline.

In [6]:
pipeline.upsert(role_arn=role)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


{'PipelineArn': 'arn:aws:sagemaker:us-east-1:093729152554:pipeline/kgpipeline1631846599',
 'ResponseMetadata': {'RequestId': 'd573a586-d7fc-43a6-9ab3-15971a1482f7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd573a586-d7fc-43a6-9ab3-15971a1482f7',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '88',
   'date': 'Fri, 17 Sep 2021 02:43:35 GMT'},
  'RetryAttempts': 0}}

In [7]:
from pprint import pprint
import json
definition = json.loads(pipeline.definition())
pprint(definition)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


{'Metadata': {},
 'Parameters': [{'DefaultValue': 's3://sm-nlp-data/ie-baseline/raw/DuIE_2_0.zip',
                 'Name': 'InputDataset',
                 'Type': 'String'},
                {'DefaultValue': 's3://sm-nlp-data/ie-baseline/processed/',
                 'Name': 'ProcessingOutputData',
                 'Type': 'String'},
                {'DefaultValue': 1,
                 'Name': 'ProcessingInstanceCount',
                 'Type': 'Integer'},
                {'DefaultValue': 'ml.c5.2xlarge',
                 'Name': 'ProcessingInstanceType',
                 'Type': 'String'},
                {'DefaultValue': 'ml.g4dn.4xlarge',
                 'Name': 'TrainInstanceType',
                 'Type': 'String'},
                {'DefaultValue': 1,
                 'Name': 'TrainInstanceCount',
                 'Type': 'Integer'},
                {'DefaultValue': '20', 'Name': 'Epochs', 'Type': 'String'},
                {'DefaultValue': '0.005',
                 'Name': 'Lea

We'll start the pipeline, accepting all the default parameters.

Values can also be passed into these pipeline parameters on starting of the pipeline, and will be covered later. 

In [8]:
execution = pipeline.start(
    parameters=dict(
        InputDataset=raw_input_data_s3_uri,
        ProcessingOutputData=f"s3://{default_bucket}/ie-baseline/processed/",
        BatchData=f"s3://{default_bucket}/psudo/psudo.json",
        NeptuneClusterIdentifier='kg-neptune',
        NeptuneInstanceSuffix='instance-1',
        IamLoadFromS3RoleName='NeptuneLoadFromS3'
    )
)

### Pipeline Operations: examining and waiting for pipeline execution

Now we describe execution instance and list the steps in the execution to find out more about the execution.

In [8]:
pipeline.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:093729152554:pipeline/kgpipeline1631699792',
 'PipelineName': 'KGPipeline1631699792',
 'PipelineDisplayName': 'KGPipeline1631699792',
 'PipelineDefinition': '{"Version": "2020-12-01", "Metadata": {}, "Parameters": [{"Name": "InputDataset", "Type": "String", "DefaultValue": "s3://sm-nlp-data/ie-baseline/raw/DuIE_2_0.zip"}, {"Name": "ProcessingOutputData", "Type": "String", "DefaultValue": "s3://sm-nlp-data/ie-baseline/processed/"}, {"Name": "ProcessingInstanceCount", "Type": "Integer", "DefaultValue": 1}, {"Name": "ProcessingInstanceType", "Type": "String", "DefaultValue": "ml.c5.2xlarge"}, {"Name": "TrainInstanceType", "Type": "String", "DefaultValue": "ml.g4dn.4xlarge"}, {"Name": "TrainInstanceCount", "Type": "Integer", "DefaultValue": 1}, {"Name": "Epochs", "Type": "String", "DefaultValue": "20"}, {"Name": "LearningRate", "Type": "String", "DefaultValue": "0.005"}, {"Name": "BatchSize", "Type": "String", "DefaultValue": "64"}, {"Name": "Ev

In [9]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:093729152554:pipeline/kgpipeline1631699792',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:093729152554:pipeline/kgpipeline1631699792/execution/46inxjuk0iq8',
 'PipelineExecutionDisplayName': 'execution-1631699896487',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2021, 9, 15, 9, 58, 16, 388000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 9, 15, 9, 58, 16, 388000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': '4560e971-4dac-4032-991d-9c6143de204e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4560e971-4dac-4032-991d-9c6143de204e',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '501',
   'date': 'Wed, 15 Sep 2021 09:58:28 GMT'},
  'RetryAttempts': 0}}

We can wait for the execution by invoking `wait()` on the execution:

In [12]:
# execution.wait()

We can list the execution steps to check out the status and artifacts:

In [13]:
execution.list_steps()

[]

### Parameterized Executions

We can run additional executions of the pipeline specifying different pipeline parameters. The parameters argument is a dictionary whose names are the parameter names, and whose values are the primitive values to use as overrides of the defaults.

Of particular note, based on the performance of the model, we may want to kick off another pipeline execution, but this time on a compute-optimized instance type and set the model approval status automatically be "Approved". This means that the model package version generated by the `RegisterModel` step will automatically be ready for deployment through CI/CD pipelines, such as with SageMaker Projects.

In [14]:
execution = pipeline.start(
    parameters=dict(
        ProcessingInstanceType="ml.c5.xlarge",
        ModelApprovalStatus="Approved",
    )
)

In [16]:
# execution.wait()

In [ ]:
execution.list_steps()

# It is the same for the QA pipeline

In [19]:
from pipelines.qa.pipeline import get_pipeline
import time

pipeline = get_pipeline(
    region=region,
    role=role,
    default_bucket=default_bucket,
    model_package_group_name="QAModelPackageGroup",
    pipeline_name=f"qa-pipeline-{int(time.time())}",
)

definition = json.loads(pipeline.definition())
pprint(definition)

The class JsonGet has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The class JsonGet has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


{'Metadata': {},
 'Parameters': [{'DefaultValue': 's3://sm-nlp-data/nlu/data/qa_raw.zip',
                 'Name': 'InputData',
                 'Type': 'String'},
                {'DefaultValue': 's3://sm-nlp-data/nlu/data/processed/',
                 'Name': 'ProcessingOutputData',
                 'Type': 'String'},
                {'DefaultValue': '0.1',
                 'Name': 'ValidationSplit',
                 'Type': 'String'},
                {'DefaultValue': '0.1', 'Name': 'TestSplit', 'Type': 'String'},
                {'DefaultValue': 1,
                 'Name': 'ProcessingInstanceCount',
                 'Type': 'Integer'},
                {'DefaultValue': 'ml.c5.2xlarge',
                 'Name': 'ProcessingInstanceType',
                 'Type': 'String'},
                {'DefaultValue': 'ml.g4dn.4xlarge',
                 'Name': 'TrainInstanceType',
                 'Type': 'String'},
                {'DefaultValue': 1,
                 'Name': 'TrainInstanceCount',

Prepare data for the QA pipeline

In [20]:
!aws s3 cp pipelines/qa/data/qa_raw.zip s3://$default_bucket/nlu/data/qa_raw.zip

upload: pipelines/qa/data/qa_raw.zip to s3://sm-nlp-data/nlu/data/qa_raw.zip


In [22]:
response = pipeline.upsert(role_arn=role)
pipeline_arn = response["PipelineArn"]
print(pipeline_arn)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


arn:aws:sagemaker:us-east-1:093729152554:pipeline/qa-pipeline-16311008021631100802


In [23]:
execution = pipeline.start(
    parameters=dict(
        InputData=f"s3://{default_bucket}/nlu/data/qa_raw.zip",
        ProcessingOutputData=f"s3://{default_bucket}/nlu/data/processed/"
    )
)

In [24]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:093729152554:pipeline/qa-pipeline-16311008021631100802',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:093729152554:pipeline/qa-pipeline-16311008021631100802/execution/5b1f3d30mp9l',
 'PipelineExecutionDisplayName': 'execution-1631101009815',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2021, 9, 8, 11, 36, 49, 749000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 9, 8, 11, 36, 49, 749000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': 'd4f2daa7-6144-4ac5-8137-a293f5f006f2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd4f2daa7-6144-4ac5-8137-a293f5f006f2',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '537',
   'date': 'Wed, 08 Sep 2021 11:36:51 GMT'},
  'RetryAttempts': 0}}